In [127]:
import pandas as pd
import numpy as np
from pathlib import Path

def process_annotations(annotation_path):
    """
    Process sleep lab annotation CSV file into a structured DataFrame
    
    Parameters:
    annotation_path : str or Path
        
    Returns:
    pd.DataFrame
    Processed DataFrame with columns: Start, Duration [ms], Event, Type, Stage
    """

    pth_to_file = Path(annotation_path)
    df = pd.read_csv(pth_to_file.as_posix())
    df.drop(['Unnamed: 1', 'Unnamed: 2'], axis=1, inplace=True)
    df.dropna(inplace=True)
    pattern = r'Start: (.*?); Duration \[ms\]: (.*?); Event: (.*?); \((.*?)\)(?:; Stage: (.*))?'
    df[['Start', 'Duration [ms]', 'Event', 'Type', 'Stage']] = df['Event-Exportdatei'].str.extract(pattern)
    df.drop('Event-Exportdatei', axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df = df.dropna(subset=['Duration [ms]'])
    return df

def convert_duration_to_indices(df, params):
    """
    Convert duration from milliseconds to number of indices based on radar sampling rate
    
    Parameters:
    df: DataFrame with 'Duration [ms]' column
    params: Dictionary containing parameters including 'fps' key
    
    Returns:
    DataFrame with new column 'Duration_indices'
    """
    fps_respiration = params['fps']
    df['Duration_indices'] = (df['Duration [ms]'].astype(float) / 1000 * fps_respiration).astype(int)
    return df



def calculate_end_times(df):
    """
    Calculate end times by adding duration to start times
    
    Parameters:
    df : pd.DataFrame
        DataFrame with 'Start' column (time strings) and 'Duration [ms]' column
        
    Returns:
    pd.DataFrame
        DataFrame with new 'End' column containing end times
    """
    start_times = pd.to_datetime(df['Start'], format='%H:%M:%S')
    duration_seconds = (df['Duration [ms]'].astype(float) / 1000).round().astype(int)
    end_times = start_times + pd.to_timedelta(duration_seconds, unit='s')
    df['End'] = end_times.dt.strftime('%H:%M:%S')
    return df

def extract_recording_date(annotation_path):
    """Extract recording date from annotations CSV file.
    Args:
        annotation_path (str|Path): Path to annotations file
    Returns:
        str: Recording date (DD.MM.YYYY)
    """
    df = pd.read_csv(annotation_path, nrows=50)
    date_row_idx = df[df.iloc[:, 0] == '[Rec. date:]'].index
    if date_row_idx.empty:
        raise ValueError("Recording date not found in file")
    
    date_str = df.iloc[date_row_idx[0] + 1, 0]
    recording_date = pd.to_datetime(date_str, format='%d.%m.%Y')
    return recording_date

In [ ]:
params = {'fps': 200}  
df = process_annotations('/Users/avitalv/Downloads/0010.csv')
df = convert_duration_to_indices(df, params)
df = calculate_end_times(df)
recording_date = extract_recording_date('/Users/avitalv/Downloads/0010.csv')

In [105]:
study = 'Sleep Lab'
from aws_manager import S3Manager, DBManagerSystem
import yaml
import pandas as pd

def fetch_motion_results(db_manager, session_id):
    query = f"SELECT * FROM MotionResult WHERE SessionID = {session_id}"
    return pd.read_sql(query, db_manager.engine)

def fetch_study_sessions(db_manager, study):
    query = f"""
    SELECT Patient.PatientStudyName, Patient.EnrollmentTime,
    Session.* FROM Session 
    JOIN Patient ON Session.PatientID = Patient.ID 
    JOIN Study ON Patient.StudyID = Study.ID 
    WHERE Study.Name = '{study}'
    """
    return pd.read_sql(query, db_manager.engine)


def load_config():
    with open('parameters.yaml', 'r') as file:
        config = yaml.safe_load(file)
    return config

S3_CONFIG = load_config()['Cloud']['s3']
DB_CONFIG = load_config()['Cloud']['DB_system']

s3_manager = S3Manager(
    access_key_id=S3_CONFIG['access_key_id'],
    secret_access_key=S3_CONFIG['secret_access_key'],
    region=S3_CONFIG['region'],
    bucket_name=S3_CONFIG['bucket_name']
)

db_manager = DBManagerSystem(
    host=DB_CONFIG['host'],
    
    user=DB_CONFIG['user'],
    password=DB_CONFIG['password'],
    database=DB_CONFIG['database']
)

In [144]:
session_df = fetch_study_sessions(db_manager, study)
matching_session = session_df[session_df['StartTime'].dt.date == recording_date.date()]
session_id = matching_session['ID'].item()



In [145]:
session_id

386

In [108]:
session_ids = session_df['ID'].tolist()
print(session_ids)

[299, 375, 379, 380, 381, 382, 384, 385, 386, 387, 388]
